In [38]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def load_json_file(filename):
    """Carga un archivo JSON y maneja posibles errores."""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {filename}")
        return None
    except json.JSONDecodeError:
        print(f"Error: El archivo {filename} no es un JSON válido")
        return None
    except Exception as e:
        print(f"Error inesperado al cargar {filename}: {str(e)}")
        return None

def get_all_courses(profesores_input):
    """Extrae todas las asignaturas únicas con sus vacantes."""
    courses = []
    for profesor in profesores_input:
        for asignatura in profesor['Asignaturas']:
            courses.append({
                'codigo': asignatura['CodigoAsignatura'],
                'nombre': asignatura['Nombre'],
                'vacantes': asignatura['Vacantes']
            })
    return courses

def analyze_room_capacity(salas_input, courses):
    """Analiza qué asignaturas podrían ser alojadas óptimamente en cada sala."""
    try:
        room_stats = {}
        
        # Para cada sala
        for sala in salas_input:
            codigo = sala['Codigo']
            capacidad = sala['Capacidad']
            optimas = 0
            no_optimas = 0
                
            # Comparar con cada asignatura
            for course in courses:
                if capacidad >= course['vacantes']:
                       optimas += 1
                else:
                    no_optimas += 1
                
            room_stats[codigo] = {
                'capacidad': capacidad,
                'optimas': optimas,
                'no_optimas': no_optimas,
                'total': optimas + no_optimas
            }
        
        return room_stats
    
    except Exception as e:
        print(f"Error al analizar la capacidad de salas: {str(e)}")
        return None

def create_horizontal_bar_chart(room_stats):
    """Crea un gráfico de barras horizontales compuesto con mayor grosor y separación."""

    # Convertir a DataFrame
    df = pd.DataFrame.from_dict(room_stats, orient='index')

    # Calcular porcentajes
    df['pct_optimas'] = (df['optimas'] / df['total'] * 100).round(2)
    df['pct_no_optimas'] = (df['no_optimas'] / df['total'] * 100).round(2)

    # Ordenar por porcentaje de asignaturas óptimas de mayor a menor
    df_sorted = df.sort_values('pct_optimas', ascending=True)

    # Crear gráfico
    fig, ax = plt.subplots(figsize=(15, 30))

    # Posición de las barras con espaciado
    y_pos = np.arange(len(df_sorted)) * 1.5  # Espaciado entre filas

    # Crear barras con mayor grosor
    bar_height = 1  # Ajuste del grosor de las barras
    bars1 = ax.barh(y_pos, df_sorted['pct_optimas'], label='Óptimas', color='#00629B', height=bar_height)
    bars2 = ax.barh(y_pos, df_sorted['pct_no_optimas'], left=df_sorted['pct_optimas'],
                    label='No Óptimas', color='#A9A9A9', height=bar_height)

    # Configurar aspecto
    ax.set_yticks(y_pos)
    ax.set_yticklabels([f"{idx} ({row['capacidad']})" for idx, row in df_sorted.iterrows()])

    # Eliminar bordes superior y derecho
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Añadir etiquetas en las barras
    for i, (opt, no_opt) in enumerate(zip(df_sorted['pct_optimas'], df_sorted['pct_no_optimas'])):
        # Etiqueta para óptimas
        if opt > 0:
            ax.text(opt / 2, y_pos[i], f"{opt}%", ha='center', va='center', color='white', fontsize=10)

        # Etiqueta para no óptimas
        if no_opt > 0:
            ax.text(opt + no_opt / 2, y_pos[i], f"{no_opt}%", ha='center', va='center', color='black', fontsize=10)

    # Títulos y etiquetas
    plt.title('Porcentaje de Asignaturas que una Sala Puede Alojar de Manera Óptima', fontsize=14)
    plt.xlabel('Porcentaje de Asignaturas', fontsize=12)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)

    # Ajustar diseño y guardar
    plt.tight_layout()
    plt.savefig('capacidad_salas.png', bbox_inches='tight')
    plt.close()

    return True

def create_summary_table(room_stats):
    """Crea y muestra una tabla resumen de las estadísticas."""
    try:
        # Crear DataFrame
        df = pd.DataFrame.from_dict(room_stats, orient='index')
        
        # Calcular porcentajes
        df['pct_optimas'] = (df['optimas'] / df['total'] * 100).round(2)
        df['pct_no_optimas'] = (df['no_optimas'] / df['total'] * 100).round(2)
        
        # Ordenar por cantidad de asignaturas óptimas
        df_sorted = df.sort_values('optimas', ascending=False)
        
        # Mostrar tabla
        print("\nResumen de Capacidad de Salas:")
        print("\nSala | Capacidad | Optimas (%) | No Optimas (%)")
        print("-" * 50)
        
        for idx, row in df_sorted.iterrows():
            print(f"{idx:6} | {row['capacidad']:9} | {row['pct_optimas']:10.2f} | {row['pct_no_optimas']:11.2f}")
        
        return df_sorted
        
    except Exception as e:
        print(f"Error al crear la tabla resumen: {str(e)}")
        return None

def save_results(room_stats, df_summary):
    """Guarda los resultados en archivos CSV y JSON."""
    try:
        # Guardar CSV
        df_summary.to_csv('resumen_capacidad_salas.csv')
        
        # Guardar JSON
        with open('metricas_capacidad_salas.json', 'w', encoding='utf-8') as f:
            json.dump(room_stats, f, ensure_ascii=False, indent=2)
        
        return True
        
    except Exception as e:
        print(f"Error al guardar los resultados: {str(e)}")
        return False

def main():
    # 1. Cargar datos
    profesores_input = load_json_file('../../agent_input/InputOfProfesores.json')
    salas_input = load_json_file('../../agent_input/InputOfSala.json')
    
    if profesores_input is None or salas_input is None:
        return
    
    # 2. Obtener todas las asignaturas
    courses = get_all_courses(profesores_input)
    
    # 3. Analizar capacidad de salas
    room_stats = analyze_room_capacity(salas_input, courses)
    if room_stats is None:
        return
    
    # 4. Crear tabla resumen
    df_summary = create_summary_table(room_stats)
    if df_summary is None:
        return
    
    # 5. Crear visualización
    if not create_horizontal_bar_chart(room_stats):
        return
    
    # 6. Guardar resultados
    if save_results(room_stats, df_summary):
        print("\nAnálisis de capacidad de salas completado exitosamente")
    else:
        print("Error al completar el análisis de capacidad de salas")

if __name__ == "__main__":
    main()


Resumen de Capacidad de Salas:

Sala | Capacidad | Optimas (%) | No Optimas (%)
--------------------------------------------------
CRP43  |      71.0 |     100.00 |        0.00
CRP33  |      71.0 |     100.00 |        0.00
KAUS1  |      68.0 |     100.00 |        0.00
K1     |      82.0 |     100.00 |        0.00
IC4    |      77.0 |     100.00 |        0.00
V205   |      65.0 |      99.60 |        0.40
V207   |      60.0 |      98.42 |        1.58
CM3    |      50.0 |      98.02 |        1.98
CM5    |      45.0 |      94.65 |        5.35
V103   |      45.0 |      94.65 |        5.35
V102   |      45.0 |      94.65 |        5.35
E6     |      45.0 |      94.65 |        5.35
E1     |      45.0 |      94.65 |        5.35
CM4    |      45.0 |      94.65 |        5.35
IC2    |      44.0 |      93.86 |        6.14
V105   |      40.0 |      91.29 |        8.71
LC6    |      40.0 |      91.29 |        8.71
LAMB   |      40.0 |      91.29 |        8.71
LC-03  |      40.0 |      91.29 |       